# MCP Server Architecture Tutorial

This notebook demonstrates the **Model Context Protocol (MCP)** architecture with hands-on examples:

- **Server**: Provides tools and resources
- **Client Agents**: Implement business logic using server tools
- **Clear separation of concerns** for scalable AI applications

## Architecture Overview

```
CLIENT (Agents)  ←→  MCP SERVER (Tools)  ←→  RESOURCES
     │                      │                    │
  ┌─────────┐          ┌─────────┐          ┌─────────┐
  │ Facts   │          │ Web     │          │ APIs    │
  │ Sent.   │    ←→    │ Wiki    │    ←→    │ DBs     │
  │ Orch.   │          │ LLM     │          │ Files   │
  └─────────┘          └─────────┘          └─────────┘
```

## Tutorial Sections

1. **Setup & Server Connection**
2. **Direct Tool Usage**
3. **Client-side Agents**
4. **Orchestration Patterns**
5. **Best Practices**

In [1]:
# Setup: Import Client-side Agents
print("🔧 Setting up MCP Client Environment")

try:
    from client_agents import (
        MCPClient,
        FactExtractorAgent,
        SentimentAnalyzerAgent,
        OrchestrationAgent
    )
    print("✅ Client-side agents imported successfully")
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("📝 Make sure client_agents.py is in the same directory")
    print("   and GEMINI_API_KEY is set in environment")
    raise

🔧 Setting up MCP Client Environment
✅ Client-side agents imported successfully
✅ Client-side agents imported successfully


/home/sajjad/miniconda3/envs/titan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. MCP Client Setup

First, let's establish connection to the MCP server:

In [ ]:
# Initialize MCP Client
try:
    client = MCPClient()
    print("✅ MCP Client initialized successfully")
except Exception as e:
    print(f"❌ Failed to initialize MCP Client: {e}")
    print("? Make sure MCP server is running:")
    print("   python mcp_server_tutorial.py")

### Health Check

Let's verify the server is running:

In [ ]:
# Health Check
health = client.check_health()

if health.get('status') == 'healthy':
    print("✅ Server is healthy")
    print(f"   Version: {health.get('version', 'unknown')}")
else:
    print("❌ Server is not healthy")
    print("   Start server: python mcp_server_tutorial.py")

### Discover Available Tools

In [ ]:
# Discover Available Tools
capabilities = client.get_capabilities()

if capabilities.get('tools'):
    tools = capabilities['tools']
    print(f"✅ Found {len(tools)} tools:")
    
    for tool_name, tool_info in tools.items():
        print(f"   🔧 {tool_name}: {tool_info.get('description', 'No description')}")
        
        # Show required parameters
        required = tool_info.get('required_params', [])
        if required:
            print(f"      Required: {', '.join(required)}")
else:
    print("❌ No tools available")

### Test a Simple Tool

In [ ]:
# Test LLM Tool
result = client.use_tool("llm", {
    "prompt": "Say hello in a friendly way",
    "max_tokens": 50
})

if result.get('success'):
    print("✅ Tool call successful")
    print(f"Response: {result.get('result', {}).get('text', 'No response')}")
else:
    print(f"❌ Tool call failed: {result.get('error')}")

## 2. Direct Tool Usage

MCP servers provide tools that can be used directly. Each tool:
- Has a unique name and description
- Accepts structured parameters
- Returns consistent response formats

### Available Tool Categories:
- **Information Retrieval**: `web_search`, `wikipedia`
- **AI Processing**: `llm`, `summarize`
- **Data Processing**: `json_parse`, `csv_process`

Let's explore these tools:

In [ ]:
# Web Search Tool Example
search_result = client.use_tool("web_search", {
    "query": "latest AI developments 2024",
    "count": 2
})

if search_result.get('success'):
    results = search_result.get('result', {}).get('results', [])
    print(f"✅ Found {len(results)} search results")
    
    for i, result in enumerate(results, 1):
        print(f"   {i}. {result.get('title', 'No title')}")
        print(f"      URL: {result.get('url', 'No URL')}")
        print(f"      Description: {result.get('description', 'No description')[:100]}...")
        print()
else:
    print(f"❌ Search failed: {search_result.get('error')}")

In [ ]:
# Wikipedia Tool Example
wiki_result = client.use_tool("wikipedia", {
    "query": "artificial intelligence",
    "sentences": 3
})

if wiki_result.get('success'):
    wiki_data = wiki_result.get('result', {})
    print(f"✅ Wikipedia article found")
    print(f"   Title: {wiki_data.get('title', 'No title')}")
    print(f"   Summary: {wiki_data.get('summary', 'No summary')[:200]}...")
    print(f"   URL: {wiki_data.get('url', 'No URL')}")
else:
    print(f"❌ Wikipedia search failed: {wiki_result.get('error')}")

In [ ]:
# JSON Processing Tool Example
json_result = client.use_tool("json_process", {
    "json_string": '{"name": "Alice", "age": 30, "skills": ["Python", "AI"]}',
    "operation": "validate_and_format"
})

if json_result.get('success'):
    json_data = json_result.get('result', {})
    print(f"✅ JSON processing successful")
    print(f"   Valid: {json_data.get('valid', 'unknown')}")
    print(f"   Formatted: {json_data.get('formatted', 'No formatted output')}")
else:
    print(f"❌ JSON processing failed: {json_result.get('error')}")

## 3. Client-side Agents

Agents add intelligence and business logic on top of server tools. They:
- Combine multiple tools to accomplish complex tasks
- Add domain-specific knowledge and processing
- Make decisions based on context and requirements

### Our Agent Types:

1. **FactExtractorAgent**: Extracts structured facts from text
2. **SentimentAnalyzerAgent**: Analyzes emotional tone and sentiment  
3. **OrchestrationAgent**: Coordinates multiple agents for complex queries

Let's see them in action:

In [ ]:
# Initialize Agents
fact_agent = FactExtractorAgent(client)
sentiment_agent = SentimentAnalyzerAgent(client)
orchestrator = OrchestrationAgent(client)

print("? Agents initialized successfully")

### Fact Extraction Agent Example

In [ ]:
# Fact Extraction Example
sample_text = """
OpenAI released GPT-4 in March 2023. It has improved reasoning capabilities 
and can handle up to 32,000 tokens of context. The ChatGPT Plus subscription 
costs $20 per month and provides access to GPT-4.
"""

facts = fact_agent.extract_facts(sample_text)

if 'error' not in facts:
    print("✅ Fact extraction successful")
    
    extracted_facts = facts.get('facts', [])
    print(f"\n📋 Extracted Facts ({len(extracted_facts)}):")
    for i, fact in enumerate(extracted_facts, 1):
        print(f"   {i}. {fact}")
    
    # Display entities if available
    entities = facts.get('entities', [])
    if entities:
        print(f"\n🏷️ Entities: {', '.join(entities)}")
        
    # Display summary if available
    summary = facts.get('summary', '')
    if summary:
        print(f"\n📝 Summary: {summary}")
else:
    print(f"❌ Fact extraction failed: {facts.get('error')}")

### Sentiment Analysis Agent Example

In [ ]:
# Sentiment Analysis Example
review_text = "I love this product! It's absolutely amazing and works perfectly. Highly recommend!"

sentiment = sentiment_agent.analyze_sentiment(review_text)

if 'error' not in sentiment:
    print("✅ Sentiment analysis successful")
    
    sentiment_class = sentiment.get('sentiment', 'unknown')
    confidence = sentiment.get('confidence', 0)
    
    print(f"\n🎭 Sentiment: {sentiment_class.upper()}")
    print(f"📈 Confidence: {confidence:.2f} ({confidence*100:.1f}%)")
    
    # Display emotional tones if available
    tones = sentiment.get('tone', [])
    if tones:
        print(f"💭 Emotional Tones: {', '.join(tones)}")
    
    # Display justification if available
    justification = sentiment.get('justification', '')
    if justification:
        print(f"📝 Justification: {justification}")
else:
    print(f"❌ Sentiment analysis failed: {sentiment.get('error')}")

## 4. Orchestration Agent

The **OrchestrationAgent** coordinates multiple agents and tools to handle complex queries that require different types of analysis.

### Key Capabilities:
- **Query Analysis**: Understands what processing is needed
- **Agent Coordination**: Manages multiple agents simultaneously  
- **Result Synthesis**: Combines results from different sources
- **Intelligent Routing**: Decides which tools and agents to use

Let's see orchestration in action:

In [ ]:
# Orchestration Example
complex_query = "What do people think about the latest Tesla Model 3 updates?"

print(f"🎯 Complex Query: {complex_query}")
print(f"\n🔍 This query requires:")
print("   • Factual information gathering (web search)")
print("   • Opinion/sentiment analysis") 
print("   • Synthesis of multiple sources")

print(f"\n? Running orchestration...")
result = orchestrator.handle_complex_query(complex_query)

if result:
    print("✅ Orchestration completed successfully")
    
    # Display strategy used
    strategy = result.get('strategy', {})
    print(f"\n📋 Processing Strategy:")
    print(f"   • Fact extraction: {strategy.get('needs_fact_extraction', False)}")
    print(f"   • Sentiment analysis: {strategy.get('needs_sentiment_analysis', False)}")
    print(f"   • Web search: {strategy.get('needs_web_search', False)}")
    
    # Display synthesis
    synthesis = result.get('synthesis', {})
    comprehensive_response = synthesis.get('comprehensive_response', 'No response')
    print(f"\n💡 Comprehensive Response:")
    print(f"   {comprehensive_response}")
    
    # Display confidence
    confidence = synthesis.get('confidence', 0)
    print(f"\n📊 Confidence: {confidence:.2f}")
    
    # Display sources
    sources = synthesis.get('sources_used', [])
    if sources:
        print(f"📚 Sources: {', '.join(sources)}")
else:
    print("❌ Orchestration failed")

## 5. Architecture Benefits

The MCP architecture provides several key advantages:

### 🎯 **Separation of Concerns**
- **Server**: Focuses on tool provision and resource management
- **Agents**: Focus on business logic and decision-making
- **Clear boundaries**: Makes debugging and maintenance easier

### 📈 **Scalability**
- Multiple clients can share the same server
- Server tools can be distributed across instances
- Agents can be specialized and optimized independently

### 🔄 **Reusability**
- Same tools work with different agent types
- Agents can be composed in various ways
- Development effort is maximized across projects

### 🛠️ **Maintainability**
- Tool updates don't affect agent logic
- Agent improvements don't require server changes
- Modular testing and deployment strategies

### ? **Security**
- Centralized access control and authentication
- Consistent security policies across all tools
- Resource protection and isolation

## 6. Best Practices

### 🏗️ **Server Design**
- **Organize tools** into logical categories
- **Implement consistent error handling** across all tools
- **Validate parameters** before processing
- **Use structured response formats**

### 🤖 **Agent Design**
- **Single responsibility**: Each agent should focus on one domain
- **Composition over inheritance**: Build complex agents from simple ones
- **Error handling**: Implement fallback strategies
- **Stateless design**: Avoid storing state between requests

### 📊 **Monitoring & Observability**
- **Comprehensive logging**: Log all tool usage and agent activities
- **Performance metrics**: Track response times and success rates
- **Health checks**: Monitor server and agent health
- **Error tracking**: Capture and analyze failures

### 🛡️ **Security**
- **Input validation**: Sanitize all inputs before processing
- **Access control**: Implement role-based permissions
- **Rate limiting**: Prevent abuse and ensure fair usage
- **Audit logging**: Track all tool and agent usage

### 🔧 **Development**
- **Unit testing**: Test agents and tools independently
- **Integration testing**: Test complete workflows
- **Documentation**: Document all tools and agents
- **Version control**: Track changes to tools and agents

In [ ]:
# Best Practices Example: Error Handling
print("🛡️ Demonstrating Robust Error Handling")

# Test with invalid tool
result = client.use_tool("nonexistent_tool", {"param": "value"})

if not result.get('success'):
    print("✅ Server properly returned error response")
    print(f"   Error: {result.get('error', 'No error message')}")
else:
    print("❌ Server should have returned an error")

# Test with invalid parameters
result = client.use_tool("llm", {"invalid_param": "value"})

if not result.get('success'):
    print("✅ Server properly validated parameters")
    print(f"   Validation error: {result.get('error', 'No error message')}")
else:
    print("⚠️ Server accepted invalid parameters")

print("\n? Key Takeaway: Good error handling provides clear feedback and helps with debugging")

## 🎉 Conclusion

### Key Takeaways:
1. **Architecture Matters**: Proper separation enables scalable, maintainable AI applications
2. **Composition Over Complexity**: Build complex systems from simple, reusable components
3. **Monitor Everything**: Comprehensive logging and monitoring are essential
4. **Security First**: Always validate inputs and implement proper access controls

### Next Steps:
- **Extend the system**: Add new tools and agents for your specific needs
- **Production features**: Implement authentication, rate limiting, and caching
- **Advanced orchestration**: Build complex multi-step workflows
- **Integration**: Connect with web frameworks and databases